<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab6/Arch_Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_6*

---


## 环境准备

In [1]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 958, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 958 (delta 16), reused 49 (delta 9), pack-reused 900 (from 2)
Receiving objects: 100% (958/958), 62.10 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (585/585), done.


In [3]:
%cd Comp_Arch

/content/Comp_Arch


In [4]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/lab6
  remotes/origin/main


In [5]:
!git switch lab6

Filtering content: 100% (7/7), 1.24 GiB | 101.15 MiB/s, done.
Branch 'lab6' set up to track remote branch 'lab6' from 'origin'.
Switched to a new branch 'lab6'


---
## make run

In [6]:
%cd /content/Comp_Arch/lab6/llama2.c

/content/Comp_Arch/lab6/llama2.c


In [7]:
!make run

gcc -O3 -o run run.c -lm
gcc -O3 -o runq runq.c -lm


In [8]:
!./run stories15M.bin

Once upon a time, there was a girl named Lily. She loved playing outside in the sunshine and feeling the breeze on her face. One day, Lily was playing with her friend, Max, when they saw a butterfly flying around. They tried to catch it, but it was too fast. 
Lily's mom came outside and saw them struggling. She said, "Let's try to catch it together." Lily's friend was very intelligent and suggested they try to bounce a ball to the butterfly. 
They started bouncing the ball around, and the butterfly flew into the air. Lily was so happy and said, "Look at that attractive butterfly! It's so pretty!" Max agreed and said, "Yeah, it is. Let's try to catch it!" 
They tried and tried, but the butterfly was too quick and they couldn't catch it. Lily's mom said, "It's okay, Lily. Maybe we can find a way to catch the butterfly if we keep trying." Lily smiled and said, "Yes, I think so too!" And
achieved tok/s: 45.180723


In [9]:
!./run stories42M.bin

Once upon a time, there was a big brown dog named Max. Max loved to play with his toys and go for walks with his owner. One day, Max's owner said they were going to take a train to visit grandma. Max was so excited because he loved to see grandma.
As they were waiting for the train, Max saw a little girl with a red balloon. Max got curious and asked the girl, "Where are you going?" The little girl said, "I'm going to visit my grandma too! Do you want to come with us?" Max was so happy and wagged his tail.
As they got on the train, Max noticed the little girl looked fearful. He asked her, "Why are you scared?" The little girl said, "I'm scared of the loud noises on the train." Max told her, "Don't worry, I'll protect you. Let's just settle down and enjoy our trip." And so, Max and the little girl settled down and enjoyed the rest of their journey together.
achieved tok/s: 15.422953


In [10]:
!./run stories110M.bin

One day, a little boy named Tim was very hungry. He wanted to make a big cake. He asked his mom for help. "Mom, can we make a cake?" Tim said. His mom said, "Yes, we can make a cake. Let's make it together."
They started to make the cake. They put all the things in a big bowl. Tim was so excited. He could not wait to eat the cake. But when they looked at the bowl, the cake was gone! Tim was sad. "Where did the cake go?" he asked. His mom did not know.
Then, they heard a small sound. They looked around and saw their dog, Max, with cake on his face. Max had eaten the cake! Tim and his mom laughed. They were not hungry anymore. They made a new cake together and had a fun day.
achieved tok/s: 5.500395
